<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [20]:
#Start Here

import numpy as np
import gensim
import os
import re
from ast import literal_eval

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import tqdm

In [30]:
file = './data/Amazon_Review_May19.csv'
df = pd.read_csv(file, usecols = [3,6, 18, 20, 21])

df['sentiment'] = df['reviews.rating'].apply(lambda x: "positive" if x>3 else "negative" )

print(df.shape)
df.tail(3)

(28332, 6)


,name,categories,reviews.rating,reviews.text,reviews.title,sentiment
28329,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...","Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5,"Very nice for light internet browsing, keeping...",You get a lot for the price!,positive
28330,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...","Fire Tablets,Tablets,All Tablets,Amazon Tablet...",5,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,positive
28331,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...","Fire Tablets,Tablets,All Tablets,Amazon Tablet...",4,"At ninety dollars, the expectionations are low...",You get what your paying for,positive


In [31]:
# It's better to put all str columns into lower case after pd_read_csv

df['name'] = df['name'].str.lower()
df['reviews.text'] = df['reviews.text'].str.lower()
df['reviews.title'] = df['reviews.title'].str.lower()

### 01- Tokenize

In [35]:
def tokenize(text):
    """Parses a string into a list of semantic units (words)

    Args:
        text (str): The string that the function will tokenize.

    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = tokens.lower().split()
    
    return tokens

In [38]:
df['tokens'] = df['reviews.text'].apply(tokenize)

### 02- Get new DataFrame after tokenization on column : reviews.text

In [39]:
df.head()

,name,categories,reviews.rating,reviews.text,reviews.title,sentiment,tokens
0,amazonbasics aaa performance alkaline batterie...,"AA,AAA,Health,Electronics,Health & Household,C...",3,i order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,negative,"[i, order, 3, of, them, and, one, of, the, ite..."
1,amazonbasics aaa performance alkaline batterie...,"AA,AAA,Health,Electronics,Health & Household,C...",4,bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,positive,"[bulk, is, always, the, less, expensive, way, ..."
2,amazonbasics aaa performance alkaline batterie...,"AA,AAA,Health,Electronics,Health & Household,C...",5,well they are not duracell but for the price i...,... are not duracell but for the price i am ha...,positive,"[well, they, are, not, duracell, but, for, the..."
3,amazonbasics aaa performance alkaline batterie...,"AA,AAA,Health,Electronics,Health & Household,C...",5,seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,positive,"[seem, to, work, as, well, as, name, brand, ba..."
4,amazonbasics aaa performance alkaline batterie...,"AA,AAA,Health,Electronics,Health & Household,C...",5,these batteries are very long lasting the pric...,... batteries are very long lasting the price ...,positive,"[these, batteries, are, very, long, lasting, t..."


In [41]:
amz = df[['reviews.text', 'sentiment', 'tokens']]
amz

,reviews.text,sentiment,tokens
0,i order 3 of them and one of the item is bad q...,negative,"[i, order, 3, of, them, and, one, of, the, ite..."
1,bulk is always the less expensive way to go fo...,positive,"[bulk, is, always, the, less, expensive, way, ..."
2,well they are not duracell but for the price i...,positive,"[well, they, are, not, duracell, but, for, the..."
3,seem to work as well as name brand batteries a...,positive,"[seem, to, work, as, well, as, name, brand, ba..."
4,these batteries are very long lasting the pric...,positive,"[these, batteries, are, very, long, lasting, t..."
...,...,...,...
28327,i got 2 of these for my 8 yr old twins. my 11 ...,positive,"[i, got, 2, of, these, for, my, 8, yr, old, tw..."
28328,i bought this for my niece for a christmas gif...,positive,"[i, bought, this, for, my, niece, for, a, chri..."
28329,"very nice for light internet browsing, keeping...",positive,"[very, nice, for, light, internet, browsing, k..."
28330,this tablet does absolutely everything i want!...,positive,"[this, tablet, does, absolutely, everything, i..."


### 03- Stopwords removal

In [58]:
# stop_words = [
#     'I',
#     'this',
#     'film', 
#     'actor',
#     'actress',
#     'films',
# ]

from nltk.corpus import stopwords
stops = stopwords.words('english')



import spacy
nlp = spacy.load('en_core_web_lg')

In [59]:
# stops = nlp.Defaults.stop_words.union(stop_words)

def remove_stops(_list_):
    
    keywords = []  
    for keyword in _list_:        
        phrase = []        
        words = keyword.split()
        
        for word in words:
            if word in stops:
                pass
            else: 
                phrase.append(word)            
        phrase = ' '.join(phrase)
        
        if len(phrase) > 0:
            keywords.append(phrase)
    
    return keywords

In [60]:
from tqdm import tqdm

tqdm.pandas()

amz['tokens1'] = amz['tokens'].progress_apply(remove_stops)

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|█████████████████████████████████████████████████████████████████████████| 28332/28332 [00:01<00:00, 20734.50it/s]
C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### 04- Get a new DataFrame: amz

In [62]:
amz.head()

,reviews.text,sentiment,tokens,tokens1
0,i order 3 of them and one of the item is bad q...,negative,"[i, order, 3, of, them, and, one, of, the, ite...","[order, 3, one, item, bad, quality, missing, b..."
1,bulk is always the less expensive way to go fo...,positive,"[bulk, is, always, the, less, expensive, way, ...","[bulk, always, less, expensive, way, go, produ..."
2,well they are not duracell but for the price i...,positive,"[well, they, are, not, duracell, but, for, the...","[well, duracell, price, happy]"
3,seem to work as well as name brand batteries a...,positive,"[seem, to, work, as, well, as, name, brand, ba...","[seem, work, well, name, brand, batteries, muc..."
4,these batteries are very long lasting the pric...,positive,"[these, batteries, are, very, long, lasting, t...","[batteries, long, lasting, price, great]"


### 05- Word Vectorization

In [ ]:
import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [89]:
# Dict representation of all our keywords
id2word = corpora.Dictionary(df['tokens1'])

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
len(id2word.keys())

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


12611

In [92]:
corpus = [id2word.doc2bow(text) for text in df['tokens1']]
# Try to see what is this
corpus[:5]

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1)],
 [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1)],
 [(17, 1), (18, 1), (19, 1)],
 [(11, 1), (19, 1), (20, 1), (21, 1), (22, 1)],
 [(19, 1), (20, 1), (23, 1), (24, 1), (25, 1)]]

In [93]:
# Try to see what this code/id2word is all about

print(id2word[0], id2word[1], id2word[2], id2word[3], id2word[4])

3 aluminum backup bad battery


C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 06- Select  25 Topics only ( for the sake of visualization)

In [100]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=25,
                   passes=50, 
                   workers=12
                  )

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
lda.print_topics()

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(14,
  '0.031*"card" + 0.023*"happy" + 0.023*"tablet" + 0.021*"sd" + 0.018*"charging" + 0.016*"memory" + 0.014*"bought" + 0.013*"charge" + 0.013*"storage" + 0.013*"problem"'),
 (1,
  '0.130*"battery" + 0.084*"life" + 0.034*"long" + 0.015*"lasts" + 0.014*"longer" + 0.013*"time" + 0.013*"charge" + 0.011*"amazon" + 0.011*"great" + 0.008*"thanks"'),
 (23,
  '0.048*"camera" + 0.024*"great" + 0.022*"ok" + 0.020*"service" + 0.018*"screen" + 0.017*"like" + 0.014*"tablet" + 0.013*"break" + 0.012*"case" + 0.011*"little"'),
 (6,
  '0.144*"kids" + 0.075*"tablet" + 0.073*"great" + 0.046*"good" + 0.024*"love" + 0.023*"learning" + 0.022*"use" + 0.016*"recommend" + 0.014*"keeps" + 0.013*"product"'),
 (8,
  '0.107*"batteries" + 0.095*"good" + 0.065*"price" + 0.062*"great" + 0.047*"work" + 0.027*"value" + 0.025*"long" + 0.024*"brand" + 0.022*"deal" + 0.019*"brands"'),
 (9,
  '0.100*"size" + 0.062*"perfect" + 0.060*"tablet" + 0.039*"good" + 0.037*"great" + 0.033*"screen" + 0.025*"price" + 0.024*"fast" +

In [102]:
len(lda.print_topics())


C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


20

In [103]:
import re

words = [re.findall(r'"[^"]*"', t[1]) for t in lda.print_topics(25)]

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [104]:
topic_labels = [', '.join(t[0:5]) for t in words]

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [105]:
# for id, t in enumerate(topic): 
#     print(f"------ Topic {id} ------")
#     print(t, end="\n\n")

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [106]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [107]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\anita\Anaconda3\envs\U4-S1-NLP\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.135449 -0.203149       1        1  11.921423
8     -0.265204 -0.003617       2        1   9.487815
3      0.266191  0.025967       3        1   5.884995
10     0.031242 -0.082678       4        1   5.573659
16     0.011029  0.046672       5        1   4.519448
21    -0.115360  0.023959       6        1   4.433073
12     0.074954  0.119259       7        1   4.421069
19    -0.158807  0.208192       8        1   4.226821
2     -0.026683  0.066703       9        1   4.182906
11    -0.135621  0.092501      10        1   4.045584
22    -0.049622 -0.068681      11        1   3.487711
1     -0.108972 -0.147084      12        1   3.441811
17    -0.038366 -0.151277      13        1   3.290524
6      0.029163  0.198420      14        1   3.250303
13     0.148335  0.089319      15        1   3.195598
14    -0.009607 -0.051445      16        1   2.997387
5     -0.002226  0.099305      17        1   2.835137
7      0.156386 -0.112813      18        1   2.785091
15     0.170418 -0.001426      19        1   2.771657
9     -0.075326  0.166471      20        1   2.686369
20     0.204760 -0.024098      21        1   2.494470
24    -0.031841 -0.165588      22        1   2.326993
23    -0.001377  0.000227      23        1   2.039706
0      0.062572 -0.105167      24        1   1.888037
4     -0.000589 -0.019970      25        1   1.812412, topic_info=           Term         Freq        Total Category  logprob  loglift
20    batteries  6819.000000  6819.000000  Default  30.0000  30.0000
1003       easy  2857.000000  2857.000000  Default  29.0000  29.0000
23        great  8907.000000  8907.000000  Default  28.0000  28.0000
99          use  4313.000000  4313.000000  Default  27.0000  27.0000
4       battery  1942.000000  1942.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
610      little    40.523081   938.978441  Topic25  -5.0072   0.8676
31          buy    47.530702  2099.970403  Topic25  -4.8477   0.2222
171      pretty    34.784668   524.406831  Topic25  -5.1599   1.2974
23        great    50.949599  8907.069431  Topic25  -4.7783  -1.1533
16          way    34.719899   616.449236  Topic25  -5.1618   1.1338

[1801 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
5871      15  0.885821     1012
10684     19  0.926947    10and
10001      3  0.824230     10in
864       23  0.941012      10x
10103      8  0.788564     11yr
...      ...       ...      ...
4133      19  0.022315  youtube
2846       3  0.862831       yr
2846      17  0.004336       yr
2846      19  0.130075       yr
2287      18  0.908700     zone

[5413 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 9, 4, 11, 17, 22, 13, 20, 3, 12, 23, 2, 18, 7, 14, 15, 6, 8, 16, 10, 21, 25, 24, 1, 5])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling